In [1]:
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import Tokenizer, RegexTokenizer, NGram
from pyspark.ml.feature import MinHashLSH
import re


filepath = "/FileStore/tables/*.txt"

def flat(pair):
    tx = re.sub(r'\n\s*\n','\n',pair[1],re.MULTILINE)
    tx = re.sub("\n", "", tx)
    tx = re.sub("\r", "", tx)
    tx = re.sub(' +', ' ', tx)
    return [[[name for name in pair[0].split('/')][-1] ,tx]]
spksess = SparkSession \
        .builder \
        .appName("LSH") \
        .getOrCreate()
spkcontx = spksess.sparkContext

df = spkcontx.wholeTextFiles(filepath).flatMap(flat)
# try to  print(df)
cook_book_df = df.toDF(['title','content'])

#try to Tokenize df 
tok = Tokenizer(inputCol="content", outputCol="words")
tokenized = tok.transform(cook_book_df)
newdata = tokenized.select("title","content", "words")

#Fit a CountVectorizerModel
cvec = CountVectorizer(inputCol="words", outputCol="features", vocabSize=50000, minDF=2)
mod = cvec.fit(newdata)
newdata = mod.transform(newdata)

# Fit a MinHashLSH mod 
mhash = MinHashLSH(inputCol="features", outputCol="hashVal", seed=12345).setNumHashTables(3)
mod = mhash.fit(newdata)
newdata = mod.transform(newdata)
print("Number of Files - ")
print(newdata.count())
print("Data types of columns - ")
print(newdata.dtypes)
newdata.show()

match_mx = mod.approxSimilarityJoin(newdata, newdata, 3, "JaccardDistance").select(col("datasetA.title")\
                    .alias("Title A"), col("datasetB.title").alias("Title B"),\
                    col("JaccardDistance")).sort(desc("JaccardDistance")).dropDuplicates(['JaccardDistance'])
threshold = 0.85
# match_mx.show()
occurences = match_mx.filter(match_mx['JaccardDistance'] < threshold)
# print(occurences.count())
print("Displaying occurences' having JaccardDistance < 0.9")
occurences.show()

Number of Files - 
76
Data types of columns - 
[('title', 'string'), ('content', 'string'), ('words', 'array<string>'), ('features', 'vector'), ('hashVal', 'array<vector>')]
+--------+--------------------+--------------------+--------------------+--------------------+
 title| content| words| features| hashVal|
+--------+--------------------+--------------------+--------------------+--------------------+
amem.txt| The American Mat...|[, the, american,...|(50000,[0,1,2,3,9...|[[4.8538975E7], [...|
amwh.txt| The American Wom...|[, the, american,...|(50000,[0,1,2,3,4...|[[146316.0], [568...|
army.txt| Manual For Army ...|[, manual, for, a...|(50000,[0,1,2,3,4...|[[106485.0], [430...|
aunt.txt| "Aunt Babette's"...|[, "aunt, babette...|(50000,[0,1,2,3,4...|[[294644.0], [659...|
bart.txt| THE IDEAL BARTEN...|[, the, ideal, ba...|(50000,[0,1,2,3,4...|[[482803.0], [558...|
beec.txt| A bookplate illu...|[, a, bookplate, ...|(50000,[0,1,2,3,4...|[[146316.0], [568...|
blue.txt| THE BLUE GRASS C...|[, the, blue, gra...|(50000,[0,1,2,3,4...|[[294644.0], [659...|
bost.txt| THE BOSTON COOKI...|[, the, boston, c...|(50000,[0,1,2,3,4...|[[72152.0], [9976...|
brkf.txt| Breakfast, Lunch...|[, breakfast,, lu...|(50000,[0,1,2,3,4...|[[106485.0], [157...|
buck.txt| Practical Housek...|[, practical, hou...|(50000,[0,1,2,3,4...|[[294644.0], [138...|
cclu.txt| Cooking in old C...|[, cooking, in, o...|(50000,[0,1,2,3,4...|[[32321.0], [1333...|
chas.txt| Dr. Chase's Reci...|[, dr., chase's, ...|(50000,[0,1,2,3,4...|[[294644.0], [157...|
chin.txt| Chinese-Japanese...|[, chinese-japane...|(50000,[0,1,2,3,4...|[[482803.0], [329...|
choc.txt| Chocolate and Co...|[, chocolate, and...|(50000,[0,1,2,3,4...|[[482803.0], [997...|
comm.txt| Common Sense in ...|[, common, sense,...|(50000,[0,1,2,3,4...|[[106485.0], [568...|
conf.txt| The Complete Con...|[, the, complete,...|(50000,[0,1,2,3,4...|[[482803.0], [157...|
coow.txt| The Cook's Own B...|[, the, cook's, o...|(50000,[0,1,2,3,4...|[[482803.0], [157...|
creo.txt| La Cuisine Creol...|[, la, cuisine, c...|(50000,[0,1,2,3,4...|[[294644.0], [430...|
dcvb.txt| Directions for C...|[, directions, fo...|(50000,[0,1,2,3,4...|[[482803.0], [430...|
dish.txt| Dishes &amp; Bev...|[, dishes, &amp;,...|(50000,[0,1,2,3,4...|[[72152.0], [1575...|
+--------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

Displaying occurences' having JaccardDistance < 0.9